# Spectroscopic Classification

This notebook assigns Branch et al. 2006 style subtypes to SDSS spectra.

In [ ]:
import sys
from pathlib import Path

import numpy as np
from astropy.table import Table
from matplotlib import pyplot as plt
from sndata.sdss import sako18spec

sys.path.insert(0, '../')
from phot_class.spectra import tabulate_spectral_properties, dust_map

sako18spec.download_module_data()

# Output directory for figures
fig_dir = Path('./notebook_figs/pew_measurements')
fig_dir.mkdir(exist_ok=True, parents=True)


We start by reading in spectroscopic measurements from the analysis pipeline. We include results determined using a range of resampling sizes.

In [ ]:
def read_spec_results(path):
    """Read in spectroscopic measurements from the analysis pipeline
    
    Args:
        path (str): The path of the ecsv file to read
        
    Returns:
        A Pandas DataFrame
    """
    
    spec_class = Table.read(path).to_pandas()
    spec_class.set_index(['feat_name', 'obj_id', 'date'], inplace=True)

    spec_summary = sako18spec.load_table(9).to_pandas()
    spec_summary = spec_summary[spec_summary['Type'] != 'Gal']
    spec_summary.drop_duplicates('CID', inplace=True)

    spec_summary.rename(columns={'CID': 'obj_id', 'Date': 'date'}, inplace=True)
    spec_summary.set_index(['obj_id'], inplace=True)

    combined_spec_data = spec_class.join(spec_summary)
    return combined_spec_data


In [ ]:
results_dir = Path('/Users/daniel/Github/Photometric-Classification/results/')
n0_spec = read_spec_results(results_dir / 'spec_class/sdss_sako18spec_3_1_0.ecsv')
n2_spec = read_spec_results(results_dir / 'spec_class/sdss_sako18spec_3_1_2.ecsv')
n5_spec = read_spec_results(results_dir / 'spec_class/sdss_sako18spec_3_1_5.ecsv')


In [ ]:
n5_spec.head()

Next we plot the ratio of the Si features.

In [ ]:
def plot_si_ratio(spec_data):
    """Plot the pW6 vs pW7 silicon pEw ratios
    
    Args:
        spec_data (DataFrame): Measurements from the analysis pipeline
        
    Returns:
        A matplotlib figure
        An array of matplotlib axes
    """
    
    fig, axes = plt.subplots(1, 3, figsize=(24, 8), sharex=True, sharey=True)
    si_data = spec_data[spec_data.Type.isin(['Ia', 'Ia-pec', 'Ia?'])].loc[['pW6', 'pW7']]
    for snr, axis in zip((0, 3, 5), axes.flatten()):
        good_snr = si_data[si_data.pew / si_data.pew_samperr  > snr]

        overlap = good_snr.loc['pW6'].index.join(good_snr.loc['pW7'].index)

        pw6 = good_snr.loc['pW6'].reindex(overlap).pew
        pw6_err = good_snr.loc['pW6'].reindex(overlap).pew_samperr
        pw7 = good_snr.loc['pW7'].reindex(overlap).pew
        pw7_err = good_snr.loc['pW7'].reindex(overlap).pew_samperr
        
        # Create an array of rgb colors
        color = np.zeros((len(pw6), 4))  # Black
        color[pw6 > 30] = 0, 0, 1, 1  # Blue
        color[(pw6 < 30) & (pw7 > 105)] = 1, 0, 0, 1  # Red
        color[pw7 < 70] = 0, 128 / 255, 0, 1  # Green
        color[:, -1] = .5 # Default alpha
        
        # Define alphas based on the combined error
        if any(pw6_err):
            total_err = pw6_err
            alpha = 1 - (total_err / max(total_err))
            alpha[alpha < .5] = .25  # So you can still see the faintest points
            color[:, -1] = alpha

        axis.errorbar(x=pw7, y=pw6, xerr=pw7_err, yerr=pw6_err, linestyle='', 
                      ecolor='grey', color='grey', alpha=.3, zorder=0)
        
        axis.scatter(pw7, pw6, color=color, zorder=10)
        axis.set_xlabel('Si ii λ6355')
        axis.set_title(f'SNR > {snr}')
        
    axes[0].set_ylabel('Si ii λ5972')
        
    return fig, axes


In [ ]:
data_frames = (n0_spec, n2_spec, n5_spec)

for num_iters, specal_data in zip((0, 2, 5), data_frames):
    fig, axes = plot_si_ratio(specal_data)
    fig.suptitle(f'{num_iters} Sampling Iterations')
    axes[0].set_xlim(-25, 700)
    axes[0].set_ylim(-25, 300)
    plt.savefig(fig_dir / f'{num_iters}_iterations.pdf')
    plt.show()
